<a href="https://colab.research.google.com/github/skywalker0803r/c620/blob/main/notebook/Integration_and_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import joblib
import os
import numpy as np
import pandas as pd
!pip install autorch > log.txt
import matplotlib.pyplot as plt
import autorch
from autorch.function import sp2wt
import random
random.seed(11)
np.random.seed(11)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# columns name

In [3]:
icg_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c620_c670.pkl')
c620_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c620_col_names.pkl')
c660_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c660_col_names.pkl')
t651_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/t651_col_names.pkl')
c670_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c670_col_names.pkl')
print(icg_c.keys())
print(c620_c.keys())
print(c660_c.keys())
print(c670_c.keys())
print(t651_c.keys())

dict_keys(['x', 'y'])
dict_keys(['x41', 'case', 'yRefluxRate', 'yControl', 'yHeatDuty', 'vent_gas_x', 'distillate_x', 'sidedraw_x', 'bottoms_x', 'vent_gas_sf', 'distillate_sf', 'sidedraw_sf', 'bottoms_sf', 'density'])
dict_keys(['x41', 'case', 'yRefluxRate', 'yControl', 'yHeatDuty', 'vent_gas_x', 'distillate_x', 'sidedraw_x', 'bottoms_x', 'vent_gas_sf', 'distillate_sf', 'sidedraw_sf', 'bottoms_sf', 'density'])
dict_keys(['xTray43', 'xTray54', 'combined', 'upper_bf', 'lower_bf', 'yRefluxRate', 'yControl', 'yHeatDuty', 'distillate_x', 'bottoms_x', 'distillate_sf', 'bottoms_sf', 'density'])
dict_keys(['x41', 'MFR'])


# DataFrame

In [4]:
icg_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c620_c670.csv',index_col=0)
c620_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c620_train.csv',index_col=0)
c660_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c660_train.csv',index_col=0)
c670_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c670_train.csv',index_col=0)
t651_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/t651_train.csv',index_col=0)
idx = list(set(icg_df.index)&
      set(c620_df.index)&
      set(c660_df.index)&
      set(c670_df.index)&
      set(t651_df.index))
len(idx)

1296

In [5]:
icg_df.loc[idx].head()

,Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr,Simulation Case Conditions_Feed Rate_Feed from V615 Btm NA_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr,Simulation Case Conditions_Feed Rate_Feed from C820 Dist NA_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr,Simulation Case Conditions_Feed Rate_Feed from T651 NA_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Toluene_wt%,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw,Simulation Case Conditions_C620 Distillate Rate_m3/hr
092-026,200.0,1.116023,11.152225,48.745979,15.0,2.338977,74.302254,17.467825,41.0,0.4524,44.244236,44.786339,70.0,920.0,0.01
139-002,200.0,1.116023,11.152225,48.745979,15.0,2.338977,74.302254,17.467825,32.0,0.0911,20.854397,74.689774,90.0,980.0,0.75
038-011,188.0,1.356000,10.321001,40.887009,15.0,2.338977,74.302254,17.467825,41.0,0.2127,35.413666,57.973106,80.0,980.0,0.01
035-026,175.0,1.356000,10.321001,40.887009,15.0,2.338977,74.302254,17.467825,77.0,0.2127,35.413666,57.973106,70.0,920.0,0.01
076-017,163.0,1.116023,11.152225,48.745979,15.0,2.338977,74.302254,17.467825,72.0,0.4524,44.244228,44.786339,80.0,920.0,0.01


In [6]:
c620_df.loc[idx][c620_c['case']].head()

,Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC,Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr,Tatoray Stripper C620 Operation_Specifications_Spec 3 : Benzene in Sidedraw_wt%
092-026,36.0,0.01,70.000015
139-002,36.0,0.75,90.000008
038-011,36.0,0.01,80.000015
035-026,36.0,0.01,70.000008
076-017,36.0,0.01,79.999992


In [7]:
c660_df.loc[idx][c660_c['case']].head()

,Benzene Column C660 Operation_Specifications_Spec 2 : NA in Benzene_ppmw,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw
092-026,920.000634,5.000000
139-002,979.999511,5.000000
038-011,979.999450,9.999907
035-026,920.000744,9.999946
076-017,919.994699,4.999996


# Input data

In [8]:
icg_input = icg_df.loc[idx,icg_c['x']]
c620_Receiver_Temp = c620_df.loc[idx].filter(regex='Receiver Temp')
c620_feed = c620_df.loc[idx,c620_c['x41']]
t651_feed = t651_df.loc[idx,t651_c['x41']]

# Output data

In [9]:
c620_op = c620_df.loc[idx,c620_c['density']+c620_c['yRefluxRate']+c620_c['yHeatDuty']+c620_c['yControl']]
c620_wt = c620_df.loc[idx,c620_c['vent_gas_x']+c620_c['distillate_x']+c620_c['sidedraw_x']+c620_c['bottoms_x']]
c660_op = c660_df.loc[idx,c660_c['density']+c660_c['yRefluxRate']+c660_c['yHeatDuty']+c660_c['yControl']]
c660_wt = c660_df.loc[idx,c660_c['vent_gas_x']+c660_c['distillate_x']+c660_c['sidedraw_x']+c660_c['bottoms_x']]
c670_op = c670_df.loc[idx,c670_c['density']+c670_c['yRefluxRate']+c670_c['yHeatDuty']+c670_c['yControl']]
c670_wt = c670_df.loc[idx,c670_c['distillate_x']+c670_c['bottoms_x']]

# config

In [10]:
config = {'icg_model_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c620_icg.pkl',
      'c620_model_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c620.pkl',
      'c660_model_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c660.pkl',
      'c670_model_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c670.pkl',
      'icg_col_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c620_c670.pkl',
      'c620_col_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c620_col_names.pkl',
      'c660_col_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c660_col_names.pkl',
      'c670_col_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c670_col_names.pkl',
      'index_9999_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/index_9999.pkl',
      'index_0001_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/index_0001.pkl',
          }

# define F

In [11]:
class F(object):
  def __init__(self,config):
    self.icg_model = joblib.load(config['icg_model_path'])
    self.c620_model = joblib.load(config['c620_model_path'])
    self.c660_model = joblib.load(config['c660_model_path'])
    self.c670_model = joblib.load(config['c670_model_path'])
    self.icg_col = joblib.load(config['icg_col_path'])
    self.c620_col = joblib.load(config['c620_col_path'])
    self.c660_col = joblib.load(config['c660_col_path'])
    self.c670_col = joblib.load(config['c670_col_path'])
    self.index_9999 = joblib.load(config['index_9999_path'])
    self.index_0001 = joblib.load(config['index_0001_path'])
    self.V615_density = 0.8626
    self.C820_density = 0.8731
    self.T651_density = 0.8749
    self.Recommended_mode = False


  def ICG_loop(self,Input):
    while True:
      output = self.icg_model.predict(Input)
      dist_rate = output['Simulation Case Conditions_C620 Distillate Rate_m3/hr'].values[0]
      na_in_benzene = Input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'].values[0]
      print('current Distillate Rate_m3/hr:{} NA in Benzene_ppmw:{}'.format(dist_rate,na_in_benzene))
      if dist_rate > 0:
        return output,Input
      else:
        Input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'] -= 30
        print('NA in Benzene_ppmw -= 30')
  
  def __call__(self,icg_input,c620_Receiver_Temp,c620_feed,t651_feed,other_args={}):
    # get index
    idx = icg_input.index

    # c620 input(case&feed)
    c620_case = pd.DataFrame(index=idx,columns=self.c620_col['case'])
    c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC'] = c620_Receiver_Temp.values
    
    if self.Recommended_mode == True:
      icg_output,icg_input = self.ICG_loop(icg_input)
      c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'] = icg_output.values
    
    if self.Recommended_mode == False:
      c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'] = other_args['c620_case_Distillate_Rate'].values
    
    c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 3 : Benzene in Sidedraw_wt%'] = icg_input['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%'].values
    c620_input = c620_case.join(c620_feed)
    
    # c620 output(op&wt)
    c620_input = c620_case.join(c620_feed)
    c620_output = self.c620_model.predict(c620_input)
    c620_sp,c620_op = c620_output.iloc[:,:41*4],c620_output.iloc[:,41*4:]
    s1,s2,s3,s4 = c620_sp.iloc[:,:41].values,c620_sp.iloc[:,41:41*2].values,c620_sp.iloc[:,41*2:41*3].values,c620_sp.iloc[:,41*3:41*4].values
    w1,w2,w3,w4 = sp2wt(c620_feed,s1),sp2wt(c620_feed,s2),sp2wt(c620_feed,s3),sp2wt(c620_feed,s4)
    wt = np.hstack((w1,w2,w3,w4))
    c620_wt = pd.DataFrame(wt,index=idx,columns=self.c620_col['vent_gas_x']+self.c620_col['distillate_x']+self.c620_col['sidedraw_x']+self.c620_col['bottoms_x'])
    
    # c620 input mass flow rate m3 to ton
    V615_Btm_m3 = icg_input['Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr'].values.reshape(-1,1)
    C820_Dist_m3 = icg_input['Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr'].values.reshape(-1,1)
    V615_Btm_ton = V615_Btm_m3*self.V615_density
    C820_Dist_ton = C820_Dist_m3*self.C820_density
    c620_feed_rate_ton = V615_Btm_ton+C820_Dist_ton
    
    # c620 output mass flow ton
    c620_mf_side = np.sum(c620_feed_rate_ton*c620_feed.values*s3*0.01,axis=1,keepdims=True)
    c620_mf_bot = np.sum(c620_feed_rate_ton*c620_feed.values*s4*0.01,axis=1,keepdims=True)

    # t651 feed mass flow rate(ton)
    t651_mf = (icg_input['Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr']*self.T651_density).values.reshape(-1,1)

    # c660 input mass flow(ton)
    c660_mf = t651_mf + c620_mf_side
    t651_mf_p ,c620_mf_side_p = t651_mf/c660_mf ,c620_mf_side/c660_mf

    # c660 input(feed & case)
    c660_feed = c620_wt[self.c620_col['sidedraw_x']].values*c620_mf_side_p + t651_feed.values*t651_mf_p
    c660_feed = pd.DataFrame(c660_feed,index=idx,columns=self.c660_col['x41'])
    c660_case = pd.DataFrame(index=idx,columns=self.c660_col['case'])
    c660_case['Benzene Column C660 Operation_Specifications_Spec 2 : NA in Benzene_ppmw'] = icg_input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'].values
    if self.Recommended_mode == True:
      c660_case['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'] = 10
    if self.Recommended_mode == False:
      c660_case['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'] = other_args['c660_case_Toluene in Benzene_ppmw'].values
    c660_input = c660_case.join(c660_feed)
    
    # c660 output(op&wt)
    c660_output = self.c660_model.predict(c660_input)
    c660_sp,c660_op = c660_output.iloc[:,:41*4],c660_output.iloc[:,41*4:]
    s1,s2,s3,s4 = c660_sp.iloc[:,:41].values,c660_sp.iloc[:,41:41*2].values,c660_sp.iloc[:,41*2:41*3].values,c660_sp.iloc[:,41*3:41*4].values
    w1,w2,w3,w4 = sp2wt(c660_feed,s1),sp2wt(c660_feed,s2),sp2wt(c660_feed,s3),sp2wt(c660_feed,s4)
    wt = np.hstack((w1,w2,w3,w4))
    c660_wt = pd.DataFrame(wt,index=idx,columns=self.c660_col['vent_gas_x']+self.c660_col['distillate_x']+self.c660_col['sidedraw_x']+self.c660_col['bottoms_x'])
    
    # c660 output mass flow (ton)
    c660_mf_bot = np.sum(c660_mf*c660_feed.values*s4*0.01,axis=1,keepdims=True)
    
    # c670 input mass flow
    c670_mf = c620_mf_bot + c660_mf_bot
    c620_mf_bot_p,c660_mf_bot_p = c620_mf_bot/c670_mf , c660_mf_bot/c670_mf
    
    # c670 feed wt%
    c670_feed = c620_wt[self.c620_col['bottoms_x']].values*c620_mf_bot_p + c660_wt[self.c660_col['bottoms_x']].values*c660_mf_bot_p
    c670_feed = pd.DataFrame(c670_feed,index=idx,columns=self.c670_col['combined'])

    c670_bf = pd.DataFrame(index=idx,columns=self.c670_col['upper_bf'])
    c620_bot_x = c620_wt[self.c620_col['bottoms_x']].values
    c660_bot_x = c660_wt[self.c660_col['bottoms_x']].values
    upper_bf = (c660_bot_x*c660_mf_bot)/(c620_bot_x*c620_mf_bot+c660_bot_x*c660_mf_bot)
    upper_bf = pd.DataFrame(upper_bf,index=idx,columns=self.c670_col['upper_bf'])
    upper_bf[list(set(self.index_9999)&set(upper_bf.columns))] = 0.9999
    upper_bf[list(set(self.index_0001)&set(upper_bf.columns))] = 0.0001
    
    # c670 input (feed%bf)
    c670_input = c670_feed.join(upper_bf)
    c670_output = self.c670_model.predict(c670_input)
    c670_sf,c670_op = c670_output.iloc[:,:41*2],c670_output.iloc[:,41*2:]
    s1 = c670_sf[self.c670_col['distillate_sf']].values
    s2 = c670_sf[self.c670_col['bottoms_sf']].values
    w1 = sp2wt(c670_feed,s1)
    w2 = sp2wt(c670_feed,s2)
    c670_wt = np.hstack((w1,w2))
    c670_wt = pd.DataFrame(c670_wt,index = idx,columns=self.c670_col['distillate_x']+self.c670_col['bottoms_x'])
    
    return c620_wt,c620_op,c660_wt,c660_op,c670_wt,c670_op

# 試算模式測試

In [12]:
f = F(config)
f.Recommended_mode = False

In [13]:
commom_idx = list(set(icg_df.index)&
      set(c620_df.index)&
      set(c660_df.index)&
      set(c670_df.index)&
      set(t651_df.index))
idx = np.random.choice(commom_idx,size=100,replace=False,p=None)

# minibatch input 
icg_input = icg_input.loc[idx]
c620_Receiver_Temp = c620_Receiver_Temp.loc[idx]
c620_dist_rate = c620_df.loc[idx,'Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr']
c660_case_Toluene_in_Benzene_ppmw = c660_df.loc[idx,'Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw']
c620_feed = c620_feed.loc[idx]
t651_feed = t651_feed.loc[idx]

# minibatch output 
c620_op = c620_op.loc[idx]
c620_wt = c620_wt.loc[idx]
c660_op = c660_op.loc[idx]
c660_wt = c660_wt.loc[idx]
c670_op = c670_op.loc[idx]
c670_wt = c670_wt.loc[idx]

# predict output

In [14]:
# 試算模式 c620_case_Distillate_Rate 和 c660_case_Toluene in Benzene_ppmw 需要使用者輸入
other_args = {'c620_case_Distillate_Rate':c620_dist_rate,
              'c660_case_Toluene in Benzene_ppmw':c660_case_Toluene_in_Benzene_ppmw,
              }
c620_wt_,c620_op_,c660_wt_,c660_op_,c670_wt_,c670_op_ = f(icg_input,c620_Receiver_Temp,c620_feed,t651_feed,other_args)

# c620 show_metrics

In [15]:
f.c620_model.show_metrics(c620_wt,c620_wt_,e=2e-2)

,R2,MSE,MAPE
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Hydrogen_wt%,1,0,NaN
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Methane_wt%,0.998752,1.42154e-05,0.171386
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Ethane_wt%,0.998629,0.003795,0.126248
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Propane_wt%,0.991609,0.0011891,0.0536893
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_n-Butane_wt%,0.999016,0.000987112,0.287108
...,...,...,...
Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_n-Pentylbenzene_wt%,0.999998,5.87262e-09,0.0151719
Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_n-Hexylbenzene_wt%,0.999998,1.08274e-09,0.0151733
Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_Nitrogen_wt%,1,0,NaN
Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_Oxygen_wt%,1,0,NaN


In [16]:
f.c620_model.show_metrics(c620_op,c620_op_,e=2e-2)

,R2,MSE,MAPE
Density_Feed Properties,0.999969,4.40599e-11,0.000575487
Density_Vent Gas Production Rate and Composition,0.990206,1.34605e-07,0.0722198
Density_Distillate Production Rate and Composition,0.997128,2.19086e-07,0.0463798
Density_Sidedraw Production Rate and Composition,0.999855,1.79225e-10,0.00126409
Density_Bottoms Production Rate and Composition,0.999583,6.24707e-12,0.000230574
Tatoray Stripper C620 Operation_Yield Summary_Reflux Rate_m3/hr,0.999326,0.0643015,0.161843
Tatoray Stripper C620 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,0.999435,0.00050806,0.135134
Tatoray Stripper C620 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,0.999406,0.000721498,0.139229
Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC,0.999788,0.00031966,0.00821546
Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC,0.999612,0.000356174,0.00761919


# c660 show_metrics

In [17]:
f.c660_model.show_metrics(c660_wt,c660_wt_,e=2e-2)

,R2,MSE,MAPE
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Hydrogen_wt%,1,0,NaN
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Methane_wt%,0.983479,0.00344141,2.69186
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Ethane_wt%,0.994691,0.233741,1.14436
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Propane_wt%,0.986079,0.0983153,1.01924
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_n-Butane_wt%,0.997734,0.0300885,2.0244
...,...,...,...
Benzene Column C660 Operation_Bottoms Production Rate and Composition_n-Pentylbenzene_wt%,0.999937,9.31437e-28,NaN
Benzene Column C660 Operation_Bottoms Production Rate and Composition_n-Hexylbenzene_wt%,0.999944,1.00628e-33,NaN
Benzene Column C660 Operation_Bottoms Production Rate and Composition_Nitrogen_wt%,1,0,NaN
Benzene Column C660 Operation_Bottoms Production Rate and Composition_Oxygen_wt%,1,0,NaN


In [18]:
f.c660_model.show_metrics(c660_op,c660_op_,e=2e-2)

,R2,MSE,MAPE
Density_Feed Properties,0.996133,2.94533e-09,0.00487749
Density_Vent Gas Production Rate and Composition,0.99909,3.3015e-07,0.0832019
Density_Distillate (Benzene Drag) Production Rate and Composition,0.998316,1.41733e-07,0.0337997
Density_Sidedraw (Benzene )Production Rate and Composition,0.97264,1.85684e-12,0.000121101
Density_Bottoms Production Rate and Composition,0.999192,6.0728e-11,0.000704016
Benzene Column C660 Operation_Yield Summary_Reflux Rate_m3/hr,0.998133,1.4259,0.725101
Benzene Column C660 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,0.997912,0.0120623,0.675153
Benzene Column C660 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,0.998158,0.0113959,0.68815
Benzene Column C660 Operation_Column Temp Profile_C660 Tray 6 (SD & Control)_oC,0.998011,7.51366e-05,0.00809616
Benzene Column C660 Operation_Column Temp Profile_C660 Tray 23 (Control)_oC,0.998444,0.00112383,0.0291848


# c670 show_metrics

In [19]:
f.c670_model.show_metrics(c670_wt,c670_wt_,e=2e-2)

,R2,MSE,MAPE
Toluene Column C670 Operation_Distillate Production Rate and Composition_Hydrogen_wt%,1,0,NaN
Toluene Column C670 Operation_Distillate Production Rate and Composition_Methane_wt%,0,1.04324e-12,NaN
Toluene Column C670 Operation_Distillate Production Rate and Composition_Ethane_wt%,0,4.90525e-10,NaN
Toluene Column C670 Operation_Distillate Production Rate and Composition_Propane_wt%,0,4.52695e-10,NaN
Toluene Column C670 Operation_Distillate Production Rate and Composition_n-Butane_wt%,0,6.88081e-47,NaN
...,...,...,...
Toluene Column C670 Operation_Bottoms Production Rate and Composition_n-Pentylbenzene_wt%,1,5.29013e-09,0.00908709
Toluene Column C670 Operation_Bottoms Production Rate and Composition_n-Hexylbenzene_wt%,1,1.01894e-09,0.00919042
Toluene Column C670 Operation_Bottoms Production Rate and Composition_Nitrogen_wt%,1,0,NaN
Toluene Column C670 Operation_Bottoms Production Rate and Composition_Oxygen_wt%,1,0,NaN


In [20]:
f.c670_model.show_metrics(c670_op,c670_op_,e=2e-2)

,R2,MSE,MAPE
Density_Distillate Production Rate and Composition,0.998151,1.54688e-12,0.000113575
Density_Bottoms Production Rate and Composition,0.999093,1.59236e-10,0.0011039
Toluene Column C670 Operation_Yield \nSummary_Reflux Rate_m3/hr,0.994529,3.14013,0.531158
Toluene Column C670 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,0.994654,0.0402885,0.539808
Toluene Column C670 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,0.994216,0.0424027,0.5391
Toluene Column C670 Operation_Column Temp Profile_C670 Tray 24 (Control)_oC,0.996552,0.000196194,0.00599463
Toluene Column C670 Operation_Column Temp Profile_C670 Btm Temp (Control)_oC,0.998131,0.00431257,0.0235868
AVG,0.996475,0.461048,0.234409


# 推薦模式 測試

In [21]:
# change mode
f.Recommended_mode = True
f.Recommended_mode

True

In [22]:
# select one sample c620_side == 70 & NA in Benzene == 980
cond = (icg_input['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']==70)&(icg_input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']==980)
one_sample = icg_input.loc[cond].sample(1)
one_sample

,Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr,Simulation Case Conditions_Feed Rate_Feed from V615 Btm NA_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr,Simulation Case Conditions_Feed Rate_Feed from C820 Dist NA_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr,Simulation Case Conditions_Feed Rate_Feed from T651 NA_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Toluene_wt%,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw
144-020,200.0,1.116023,11.152225,48.745979,15.0,2.338977,74.302254,17.467825,92.0,0.0911,20.854397,74.689774,70.0,980.0


In [23]:
idx = one_sample.index
print(idx)

Index(['144-020'], dtype='object')


In [24]:
demo = {'icg_input':icg_input.loc[idx],
      'c620_Receiver_Temp':c620_Receiver_Temp.loc[idx],
      'c620_case_Distillate_Rate':c620_df.loc[idx,['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr']],
      'c660_case_Toluene_in_Benzene_ppmw':c660_df.loc[idx,['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw']],
      'c620_feed':c620_feed.loc[idx],
      't651_feed':t651_feed.loc[idx],
      'c620_op':c620_op.loc[idx],
      'c620_wt':c620_wt.loc[idx],
      'c660_op':c660_op.loc[idx],
      'c660_wt':c660_wt.loc[idx],
      'c670_op':c670_op.loc[idx],
      'c670_wt':c670_wt.loc[idx],
      }

In [25]:
icg_input = demo['icg_input']
c620_Receiver_Temp = demo['c620_Receiver_Temp']
c620_dist_rate = demo['c620_case_Distillate_Rate']
c660_case_Toluene_in_Benzene_ppmw = demo['c660_case_Toluene_in_Benzene_ppmw']
c620_feed = demo['c620_feed']
t651_feed = demo['t651_feed']

In [26]:
# 推薦模式 c620_case_Distillate_Rate 來自icg輸出 c660_case_Toluene in Benzene_ppmw 固定為10 不用輸入
c620_wt_,c620_op_,c660_wt_,c660_op_,c670_wt_,c670_op_ = f(icg_input,c620_Receiver_Temp,c620_feed,t651_feed)

current Distillate Rate_m3/hr:-0.061414528638124466 NA in Benzene_ppmw:980.0
NA in Benzene_ppmw -= 30
current Distillate Rate_m3/hr:-0.023261649534106255 NA in Benzene_ppmw:950.0
NA in Benzene_ppmw -= 30
current Distillate Rate_m3/hr:-0.03711140155792236 NA in Benzene_ppmw:920.0
NA in Benzene_ppmw -= 30
current Distillate Rate_m3/hr:-0.053852349519729614 NA in Benzene_ppmw:890.0
NA in Benzene_ppmw -= 30
current Distillate Rate_m3/hr:-0.06820464134216309 NA in Benzene_ppmw:860.0
NA in Benzene_ppmw -= 30
current Distillate Rate_m3/hr:-0.08533526957035065 NA in Benzene_ppmw:830.0
NA in Benzene_ppmw -= 30
current Distillate Rate_m3/hr:-0.11258702725172043 NA in Benzene_ppmw:800.0
NA in Benzene_ppmw -= 30
current Distillate Rate_m3/hr:-0.13589732348918915 NA in Benzene_ppmw:770.0
NA in Benzene_ppmw -= 30
current Distillate Rate_m3/hr:-0.1451788991689682 NA in Benzene_ppmw:740.0
NA in Benzene_ppmw -= 30
current Distillate Rate_m3/hr:-0.15091852843761444 NA in Benzene_ppmw:710.0
NA in Benzene

In [27]:
import joblib
joblib.dump(demo,"/content/drive/MyDrive/台塑輕油案子/data/c620/demo/demo.pkl")

['/content/drive/MyDrive/台塑輕油案子/data/c620/demo/demo.pkl']